In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy_sentiws import spaCySentiWS

In [2]:
df = pd.read_csv('../../data/p_content.csv')
df.head()

,ID_GodotObject,author,source,publishingDate,content,titletext,merged_text,tokens
0,2000115059032,(keine Angabe),Redaktion,2020-02-26 16:50:27.000,﻿20sars-cov-2maskenpflicht medizinisch Persona...,Maskenpflicht medizinisch Personal Umgang Coro...,﻿20sars-cov-2maskenpflicht medizinisch_Persona...,"['\ufeff20sars-cov-2maskenpflicht', 'medizinis..."
1,2000116305030,(keine Angabe),Redaktion,2020-03-30 05:36:31.240,﻿20 Coronavirus schrittweise Einführung Masken...,schrittweise Einführung Maskenpflicht Öffentli...,﻿20_Coronavirus schrittweise Einführung Masken...,"['\ufeff20_Coronavirus', 'schrittweise', 'Einf..."
2,2000116325081,(keine Angabe),Redaktion,2020-03-30 13:17:32.120,﻿20kein Ende Sicht Regierung setzen Maske bei ...,Regierung setzen Maske bei Einkauf Test freis...,﻿20kein Ende Sicht Regierung setzen Maske bei ...,"['\ufeff20kein', 'Ende', 'Sicht', 'Regierung',..."
3,2000116346340,(keine Angabe),Redaktion,2020-03-31 05:34:39.260,﻿20 Coronavirus Maskenpflicht Supermarkt späte...,Maskenpflicht Supermarkt spätestens ab Montag ...,﻿20_Coronavirus Maskenpflicht Supermarkt späte...,"['\ufeff20_Coronavirus', 'Maskenpflicht', 'Sup..."
4,2000116371728,(keine Angabe),Redaktion,2020-03-31 16:15:23.100,﻿20 Coronavirus Sonderbeauftragter Clemens Aue...,Sonderbeauftragter Clemens Auer italienisch ...,﻿20_Coronavirus Sonderbeauftragter Clemens Aue...,"['\ufeff20_Coronavirus', 'Sonderbeauftragter',..."


https://github.com/Liebeck/spacy-sentiws

In [3]:
nlp = spacy.load('de_core_news_md')
nlp.add_pipe('sentiws',config={'sentiws_path': '../../data/sentiws/'})
doc = nlp('Die Dummheit der Unterwerfung blüht in hübschen Farben.')

# Initialize a variable to store the overall sentiment score
sentiment_score = 0
for token in doc:
    print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))
    sentiment_score += token._.sentiws or 0
    
print(sentiment_score)

Die, None, DET
Dummheit, -0.4877, NOUN
der, None, DET
Unterwerfung, -0.3279, NOUN
blüht, 0.2028, VERB
in, None, ADP
hübschen, 0.4629, ADJ
Farben, None, NOUN
., None, PUNCT
-0.14990000000000014


Now on the Whole DataFrame

In [7]:
def get_overall_sentiment(text):
    text = text.replace('_',' ')
    doc = nlp(text)
    sentiment_score = 0
    for token in doc:
        #print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))
        sentiment_score += token._.sentiws or 0
    
    return sentiment_score / len(doc)


df['sentiment_score'] = df['merged_text'].apply(lambda x: get_overall_sentiment(str(x)))
#df['sentiment_titletext'] = df['titletext'].apply(lambda x: get_overall_sentiment(str(x)))

df.head()

,ID_GodotObject,author,source,publishingDate,content,titletext,merged_text,tokens,sentiment_score
0,2000115059032,(keine Angabe),Redaktion,2020-02-26 16:50:27.000,﻿20sars-cov-2maskenpflicht medizinisch Persona...,Maskenpflicht medizinisch Personal Umgang Coro...,﻿20sars-cov-2maskenpflicht medizinisch_Persona...,"['\ufeff20sars-cov-2maskenpflicht', 'medizinis...",-0.011892
1,2000116305030,(keine Angabe),Redaktion,2020-03-30 05:36:31.240,﻿20 Coronavirus schrittweise Einführung Masken...,schrittweise Einführung Maskenpflicht Öffentli...,﻿20_Coronavirus schrittweise Einführung Masken...,"['\ufeff20_Coronavirus', 'schrittweise', 'Einf...",-0.003717
2,2000116325081,(keine Angabe),Redaktion,2020-03-30 13:17:32.120,﻿20kein Ende Sicht Regierung setzen Maske bei ...,Regierung setzen Maske bei Einkauf Test freis...,﻿20kein Ende Sicht Regierung setzen Maske bei ...,"['\ufeff20kein', 'Ende', 'Sicht', 'Regierung',...",-0.004477
3,2000116346340,(keine Angabe),Redaktion,2020-03-31 05:34:39.260,﻿20 Coronavirus Maskenpflicht Supermarkt späte...,Maskenpflicht Supermarkt spätestens ab Montag ...,﻿20_Coronavirus Maskenpflicht Supermarkt späte...,"['\ufeff20_Coronavirus', 'Maskenpflicht', 'Sup...",-0.003898
4,2000116371728,(keine Angabe),Redaktion,2020-03-31 16:15:23.100,﻿20 Coronavirus Sonderbeauftragter Clemens Aue...,Sonderbeauftragter Clemens Auer italienisch ...,﻿20_Coronavirus Sonderbeauftragter Clemens Aue...,"['\ufeff20_Coronavirus', 'Sonderbeauftragter',...",-0.003153


In [10]:
df1 = df[['publishingDate','ID_GodotObject','sentiment_score']]


df1.to_csv('../../data/feature/sentiws_content.csv', encoding='utf-8', index=False)